In [14]:
import pandas as pd
import math
import numpy as np
import random
from pulp import *

#Cargar datos de entrada, configuración de la red
df=pd.read_excel("RMC.xlsx")
red=df.to_numpy().tolist()
Tam = len(red[0])
costoN=[5,3,6,5,3,7,9,4,6]
#print(Tam)
#print(red)

#Creamos el problema
prob_rmc=LpProblem('RMC', LpMinimize)

#creamos las variables
Nodos = range(0,Tam)
Rutas = range(0,2)
arcos=[(i,j) for i in Nodos for j in Nodos]
#print(arcos)
var_ruta=LpVariable.dicts('Ruta', (Nodos,Nodos,Rutas),lowBound=0,cat='Binary')
var_nodo=LpVariable.dicts('Nodo',Nodos,lowBound=0, cat='Binary')
#print(var_ruta)

#Función objetivo
prob_rmc += lpSum(var_ruta[i][j][r]*red[i][j] for (i,j) in arcos for r in Rutas) + lpSum(var_nodo[i]*costoN[i] for i in Nodos)

#Restricciones
#origen
for r in Rutas:
  prob_rmc += lpSum(var_ruta[0][j][r] for j in Nodos) == 1

#destino
for r in Rutas:
  prob_rmc += lpSum(var_ruta[i][8][r] for i in Nodos) == 1

#balance
for r in Rutas:
  for k in Nodos:
    if k>0 and k<8:
      prob_rmc += lpSum(var_ruta[i][k][r] for i in Nodos) == lpSum(var_ruta[k][j][r] for j in Nodos)

#relación de variables
prob_rmc += var_nodo[0] == 1 #obligo a que el nodo de salida se use, los otros los analizo por la entrada
for k in Nodos:
  if k>0:
    prob_rmc += lpSum(var_ruta[i][k][r] for i in Nodos for r in Rutas) <= 999*var_nodo[k]

#rutas excluyentes
for i in Nodos:
  for k in Nodos:
    prob_rmc += lpSum(var_ruta[i][k][r] for r in Rutas) <= 1


#arcos inexistentes
#for i in origen:
#  for j in origen:
#      prob_rmc += var_ruta[i][j] <= red[i][j]

#resolvemos
status = prob_rmc.solve()
print("Status: ", status)
print("Status: ", LpStatus[prob_rmc.status])

for v in prob_rmc.variables():
    if v.value()>0:
        print(v.name,"=",v.varValue)

print("El retardo total es: " + format(value(prob_rmc.objective)))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ivanramirez/Documents/Javeriana/Optimización y Simulación /venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/z2/zx1zd8x96zl5yds5w6w3zsth0000gn/T/f557deac560e47159308065c7f2e20e8-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/z2/zx1zd8x96zl5yds5w6w3zsth0000gn/T/f557deac560e47159308065c7f2e20e8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 113 COLUMNS
At line 1216 RHS
At line 1325 BOUNDS
At line 1497 ENDATA
Problem MODEL has 108 rows, 171 columns and 575 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 54.046 - 0.00 seconds
Cgl0003I 1 fixed, 0 tightened bounds, 77 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows,